In [1]:
print("Settings.py should be e.g. flows_1")
#DESIGN PARAMETERS FOR NEURAL NETWORK
NR_LSTM_UNITS = 2 
IMAGE_INPUT_SIZE_W = 640
IMAGE_INPUT_SIZE_H = 480

IMAGE_AFTER_CONV_SIZE_W = 18
IMAGE_AFTER_CONV_SIZE_H = 13
#for 3x3 kernels, n=num_layers: len_in = 2^n*len_out + sum[i=1..n](2^i)
#CONV_LAYER_LENGTH = 5

LSTM_IO_SIZE = 18*13
LSTM_HIDDEN_SIZE = 18*13

RGB_CHANNELS = 3
TIMESTEPS = 10 # size videos
BATCH_SIZE = 5 #until now just batch_size = 1
SLICE_FROM_TIMESTEP = 1 #slices from timestep SLICE_FROM_TIMESTEP to the last one

NR_EPOCHS = 20

VALIDATION_SPLIT = 0.0 #indicated ratio of training to validation data: 0.2 -> 20% VALIDATION data
RANDOMIZED_SEED = 20
SHUFFLE_DATASET = False

learning_rate = 0.01 # reduce factos of 10 .. some epoch later.
momentum = 0.9
print("Hyperparameters defined")

Settings.py should be e.g. flows_1
Hyperparameters defined


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import os.path as path
import copy
# from torch.utils.data.sampler import SequentialSampler


from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer

import matplotlib.pyplot as plt

print("done")

done


In [3]:
#change configuration in settings.py
crop_im_size = [186,250]
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
c.config['transformers']['DimensionResize']['dimension'] = TIMESTEPS
#c.config['indexing']['people selection'] = [1]
#c.config['indexing']['sequences_selection'] = ['nm-01']
c.config['pose']['load'] = False
c.config['flow']['load'] = True
c.config['heatmaps']['load'] = False
#c.config['scenes']['sequences'] = ['nm']
#c.config['scenes']['angles'] = ['108']
c.config['dataset_output'] = {
#         'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
        'data': ['flows'],
        'label': "annotations"}
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)

loading configuration  flows_1
[OK]


# Settings 
## Dataset

In [4]:
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
print("Indices size:", len(indices))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
print("Split:", split)
if SHUFFLE_DATASET:
    np.random.seed(RANDOMIZED_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

# #

Dataset size: 135
Indices size: 135
Split: 0


## neural network

In [5]:
class TEST_CNN_LSTM(nn.Module):
    def __init__(self):
        super(TEST_CNN_LSTM, self).__init__()
        self.avialable_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.conv1 = nn.Conv2d(3,6,3) #input 640x480
        self.pool1 = nn.MaxPool2d(2,2) #input 638x478 output 319x239
        self.conv2 = nn.Conv2d(6,16,3) # input 319x239 output 317x237
        self.pool2 = nn.MaxPool2d(2,2) # input 317x237 output 158x118
        self.conv3 = nn.Conv2d(16,6,3) # input 158x118 output 156x116
        self.pool3 = nn.MaxPool2d(2,2) # input 156x116 output 78x58
        self.conv4 = nn.Conv2d(6,3,3)  # input 78x58 output 76x56
        self.pool4 = nn.MaxPool2d(2,2) # input 76x56 output 39x29
        self.conv5 = nn.Conv2d(3,1,3)  # input 39x29 output 37x27
        self.pool5 = nn.MaxPool2d(2,2) #output 37x27 output 18x13
        self.lstm1 = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            TIMESTEPS)# horizontal direction
        self.lstm2 = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            TIMESTEPS)# horizontal direction
        self.fc1 = nn.Linear(LSTM_IO_SIZE,120)
        self.fc2 = nn.Linear(120,20)
        self.fc3 = nn.Linear(20,3)
        
        #initialize hidden states of LSTM
        self.hidden = self.init_hidden()

        #print("Hidden:", _hidden)
    def init_hidden(self):
        return (torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device), 
                torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device))
    def forward(self,x):
#         print("Input list len:",len(x))
#         print("Input elemens size:", x[0].size())
#         batch_size = x[0].size()[0]

        x_arr = torch.zeros(TIMESTEPS,BATCH_SIZE,1,IMAGE_AFTER_CONV_SIZE_H,IMAGE_AFTER_CONV_SIZE_W).to(self.avialable_device)
        #print("X arr size", x_arr.size())
        #print("X size", len(x))
        for i in range(TIMESTEPS):#parallel convolutions which are later concatenated for LSTM
            x_tmp_c1 = self.pool1(F.relu(self.conv1(x[i].float())))
            x_tmp_c2 = self.pool2(F.relu(self.conv2(x_tmp_c1)))
            x_tmp_c3 = self.pool3(F.relu(self.conv3(x_tmp_c2)))
            x_tmp_c4 = self.pool4(F.relu(self.conv4(x_tmp_c3)))
            x_tmp_c5 = self.pool5(F.relu(self.conv5(x_tmp_c4)))
            x_arr[i] = x_tmp_c5 # torch.squeeze(x_tmp_c5)
        #print(x_arr)
        x, hidden = self.lstm1(x_arr.view(TIMESTEPS,BATCH_SIZE,-1), self.hidden)
        x, hidden = self.lstm2(x, self.hidden)
        # the reshaping was taken from the documentation... and makes sense
        x = x.view(TIMESTEPS,BATCH_SIZE,LSTM_HIDDEN_SIZE) #output.view(seq_len, batch, num_dir*hidden_size)
#         x = torch.squeeze(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        #x = x.squeeze(1)
        x = x.permute(1,2,0)
        #print ("Size network output", x.shape)
        return x
print("Class defined")

Class defined


In [6]:
### define and execute testing function
def test_all_preds(model):
    n_batches_test = len(test_loader)

    #Time for printing
    testing_start_time = time.time()

    print('Start testing...')
    correct = 0 
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            
            data_in = [s.to(device) for s in inputs['flows']]
            labels = labels.to(device)
            if not labels.size()[0] == BATCH_SIZE:
                # skip uncompleted batch size NN is fixed to BATCHSIZE
                continue
            outputs = model(data_in)
#             print("Out:", len(outputs), outputs.size())
#             print("Labels:", len(labels), labels.size())
            _, predicted = torch.max(outputs.data, 1)
#             print('predicted:',len(predicted),predicted.size())
            n_errors = torch.nonzero(torch.abs(labels.long() - predicted)).size(0)
            total += predicted.numel()
            # print('predicted',predicted)
            correct += predicted.numel() - n_errors
            # print('labels',labels)
    print('Accuracy {:.2f}%'.format(100*correct/total))
    print('...testing finished')
print("Definition done")

Definition done


### Test-Training

In [7]:
#TRAINING
test_net = TEST_CNN_LSTM()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
test_net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(test_net.parameters(), lr=learning_rate, momentum=momentum)


cpu


In [8]:
parameters_conv5_init = list(test_net.conv5.parameters())
bias_conv5_init = np.sort(parameters_conv5_init[1].detach().numpy())
weights_conv5_init = np.sort(parameters_conv5_init[0].detach().numpy().flatten())

parameters_fc1_init = list(test_net.fc1.parameters())
bias_fc1_init = np.sort(parameters_fc1_init[1].detach().numpy())
weights_fc1_init = np.sort(parameters_fc1_init[0].detach().numpy().flatten())

a = list(test_net.lstm2.parameters())
weights_lstm2_init = np.array([])
for item in a:
    item = item.detach().numpy().flatten()
    #print(item)
    weights_lstm2_init = np.concatenate((weights_lstm2_init,item))
print("Saved initial parameters")

Saved initial parameters


In [9]:
a = list(test_net.lstm1.parameters())
b = [item for sublist in a for item in sublist]
l = np.array([])
#print(a[0])
for item in a:
    item = item.detach().numpy().flatten()
    #print(item)
    l = np.concatenate((l,item))
print(l)
#plt.plot(l)

[-0.01398033  0.05022357 -0.01906324 ... -0.01979078  0.01589876
 -0.05671026]


### Test set and evaluate computing time etc.

In [10]:
#PREPARATION FOR TRAINING
loss_array = []
learning_rate_array = []

print('Start training...')
print("Expected loss with {} different classes and {} data elements: {}".format(3, len(dataset)-split, (len(dataset)-split)*np.log(3)))
running_loss = 0.0
#print("Data set length:", len((train_loader)), "Validation length:", len(test_loader))
print("Batch size:", BATCH_SIZE)
print("Evaluating first element...")
start_time = time.time()
i, batch = next(iter(enumerate(train_loader)))
inputs, labels = batch
data_in = [s.to(device) for s in inputs['flows']]
labels = labels.to(device)
print("Time steps:{}, input sequence length:{}".format(TIMESTEPS,len(data_in)))
#print("NN input: ",len(flows),len(flows[0]),len(flows[0][0]),len(flows[0][0][0]),len(flows[0][0][0][0]))
optimizer.zero_grad() 
outputs = test_net(data_in)
print("Expected output format: [BATCH, NR_CLASSES, TIMESTEPS]")
print("Output format:", len(outputs), outputs.size())
print("Expected label format: [BATCH, TIMESTEPS] (with int-label as each element indicating the correct one)")
print("Labels:", len(labels), labels.size())
print("Slicing loss. Using loss from:",SLICE_FROM_TIMESTEP,"to",TIMESTEPS)
#print("Labels content:", labels)
#original: loss = criterion(outputs.float(),labels.long())
loss = criterion(outputs[:,:,SLICE_FROM_TIMESTEP:TIMESTEPS].float(),labels[:,SLICE_FROM_TIMESTEP:TIMESTEPS].long())
loss.backward() 
optimizer.step()

running_loss += loss.data.item()
elapsed_time = time.time() - start_time;
loss_array.append(running_loss)
learning_rate_array.append(learning_rate)
print("Loss:{}, expected loss:{}".format(running_loss, np.log(3)))
print("Time needed:{}s".format(elapsed_time))
print("Expected loss for total training data: ", (len(dataset)-split)*np.log(3))
print("Expected training time per epoch:{} min".format(elapsed_time* len(train_loader)/60))
print("Estimated total training time:{} hours".format(elapsed_time* len(train_loader)*NR_EPOCHS/3600))

Start training...
Expected loss with 3 different classes and 135 data elements: 148.31265897019483
Batch size: 5
Evaluating first element...
Time steps:10, input sequence length:10
Expected output format: [BATCH, NR_CLASSES, TIMESTEPS]
Output format: 5 torch.Size([5, 3, 10])
Expected label format: [BATCH, TIMESTEPS] (with int-label as each element indicating the correct one)
Labels: 5 torch.Size([5, 10])
Slicing loss. Using loss from: 1 to 10
Loss:1.0581157207489014, expected loss:1.0986122886681098
Time needed:15.705131530761719s
Expected loss for total training data:  148.31265897019483
Expected training time per epoch:7.067309188842773 min
Estimated total training time:2.355769729614258 hours


### Start Training here

In [12]:
for epoch in range(NR_EPOCHS): 
    running_loss = 0.0
    start_time = time.time()
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        data_in = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        
        outputs = test_net(data_in)
        loss = criterion(outputs[:,:,SLICE_FROM_TIMESTEP:TIMESTEPS].float(),labels[:,SLICE_FROM_TIMESTEP:TIMESTEPS].long())
        loss.backward() 
        optimizer.step()
        
        running_loss += loss.data.item()

    print("Loss epoch {}: {}, took {}s".format(epoch, running_loss,time.time()-start_time))
    loss_array.append(running_loss)
    learning_rate_array.append(learning_rate)

#test_all_preds(test_net) 
print('...Training finished')
plt.plot(loss_array)

Epoch: 0
Loss epoch 0: 30.434785187244415, took 465.99632835388184s
Epoch: 1


KeyboardInterrupt: 

In [ ]:
#DYNAMIC CHANGES:
NR_EPOCHS = 20
learning_rate = 0.01
momentum = 0.9
optimizer = optim.SGD(test_net.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
#EVALUATION
print("Expected loss:{}, last loss:{}".format((len(dataset)-split)*np.log(3),loss_array[-1]))
print("Batch size:", BATCH_SIZE)
print("Sequence length:",TIMESTEPS)
print("Total epochs learnt:", len(loss_array))
plt.plot(learning_rate_array)
plt.plot(loss_array)
test_all_preds(test_net)


In [ ]:
parameters_fc1 = list(test_net.fc1.parameters())
bias_fc1 = np.sort(parameters_fc1[1].detach().numpy())
weights_fc1 = np.sort(parameters_fc1[0].detach().numpy().flatten())


a = list(test_net.lstm2.parameters())
weights_lstm2 = np.array([])
for item in a:
    item = item.detach().numpy().flatten()
    #print(item)
    weights_lstm2 = np.concatenate((weights_lstm2,item))


parameters_conv5 = list(test_net.conv5.parameters())
bias_conv5 = np.sort(parameters_conv5[1].detach().numpy())
weights_conv5 = np.sort(parameters_conv5[0].detach().numpy().flatten())
#plt.figure()
#plt.plot(range(len(weights_conv1)),weights_conv1_init,range(len(weights_conv1)),weights_conv1)
#plt.figure()
#plt.plot(range(len(bias_conv1)),bias_conv1_init,range(len(bias_conv1)),bias_conv1)

##FC layers
plt.figure()
plt.plot(range(len(weights_conv5)),weights_conv5_init-weights_conv5)#,range(len(weights_conv5)),weights_conv5)
plt.figure()
plt.plot(range(len(bias_conv5)),bias_conv5_init-bias_conv5)#,range(len(bias_conv5)),bias_conv5)
#plt.figure()
#plt.plot(range(len(weight_lstm1_0)),weight_lstm1_0_init,range(len(weight_lstm1_0)),weight_lstm1_0)


plt.figure()
plt.plot(weights_lstm2_init-weights_lstm2)
## single plots
#plt.figure()
#plt.plot(range(len(weights_conv1)),weights_conv1_init)
#plt.figure()
#plt.plot(range(len(weights_conv1)),weights_conv1)

### Expected loss for untrained set with Cross Entropy:
k = number of classes
N = number of labeled data in dataset
loss_per_prediction = -log(1/k) = log(k)
total_loss = sum(log(k)) = N*log(k)

In [ ]:
plt.clf()
plot_path = "/home/matthias/Desktop/"
loss_str = "{0:.3f}".format(loss_array[-1])
information = "flows_L" + loss_str + "_timesteps{}_nrsequences{}_batch{}".format(TIMESTEPS,dataset_size,BATCH_SIZE)
plt.xlabel('Epochs')
plt.ylabel('Loss (Cross Entropy)')
plt.plot(loss_array)
plt.title(information)
plt.savefig(plot_path + information + ".eps")
plt.clf()
plt.xlabel('Epochs')
plt.ylabel('Learning rate')
plt.plot(learning_rate_array)
plt.title(information)
plt.savefig(plot_path + information + "LR.eps")

In [ ]:
print(outputs.size())
print(outputs[:,:,5:10])
print(outputs.size()[2])
print("Loss half:", criterion(outputs[:,:,SLICE_FROM_TIMESTEP:TIMESTEPS].float(),labels[:,SLICE_FROM_TIMESTEP:TIMESTEPS].long()))
print("Loss full:", criterion(outputs.float(),labels.long()))


In [ ]:
#for param in model.parameters():
#    print(type(param.data), param.size())
#parameters_fc1 = list(test_net.fc1.parameters())
#bias = np.sort(parameters_fc1[1].detach().numpy())
#weights = np.sort(parameters_fc1[0].detach().numpy().flatten())
parameters_conv5 = list(test_net.conv5.parameters())
bias_conv5 = np.sort(parameters_conv5[1].detach().numpy())
weights_conv5 = np.sort(parameters_conv5[0].detach().numpy().flatten())

In [ ]:
parameters_lstm1 = list(test_net.lstm1.parameters())
print(len(parameters_lstm1[0]))